# ❗ 4. Transformer 기반 예측모델(EDA) ❗

## 🔧 1. 환경 설정 및 라이브러리 준비

In [1]:
# cuda 버젼에 맞게 pytorch 설정 (https://pytorch.org/get-started/previous-versions/)
# 별도로 아래의 명령어로 설치해야 함
# !pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu121

# 그 외 필요한 패키지 설치
!pip install -r "./week4_requirements.txt"


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not open requirements file: [Errno 2] No such file or directory: './week4_requirements.txt'


## 📂 2. 데이터 불러오기
----  
* pandas의 read_excel 메소드 이용
* 입력 인자  
    * sheet_name = None : 모든 시트를 불러와서 시트명이 key, 시트 내용 담긴 데이터프레임이 value인 딕셔너리 생성
        * 특정 시트만 불러오고 싶은 경우 해당 시트의 이름을 적으면 됨  
        ex) sheet_name="Data_Set1(사외)"
    * header = 1 : 열 이름이를 나타낸 행의 위치 (ex. 두번째 행이면 1로 설정(0부터 시작하기 때문))


In [1]:
import pandas as pd


excel = pd.read_excel("/DATA/SKhynix/PBL_TAT_Set_Data Rev2(사외).xlsx", sheet_name=None, header=1)

total = pd.concat([excel[sheet_name] for sheet_name in ["Data_Set1(사외)","Data_Set2(사외)"]] )

# Unnamed: 0 컬럼 제거
total.drop(columns="Unnamed: 0", inplace=True)

# total.head()



KeyboardInterrupt: 

## 결측치 확인 후 제거

* 총 729584의 행 존재
* 결측치 행 제거: y 값이 비어 있는 행
* 활용 입력 변수: x22 부터는 결측치가 거의 대부분이기 때문에, x21까지 활용하는 것으로 진행

In [ ]:
print(f"Total: {len(total)}")
total.isna().sum()

Total: 1729584


timekey_hr          0
lot_cd              0
oper_area           0
oper_group          0
oper_id             0
days                0
shift               0
x1                  0
x2                  0
x3                  0
x4                  0
x5                  0
x6                  0
x7                  0
x8                  0
x9                  0
x10                 0
x11                 0
x12                 0
x13                 0
x14                 0
x15                 0
x16                 0
x17                 0
x18                 0
x19                 0
x20                 0
x21                 0
x22           1710072
x23           1710072
x24           1710072
x25           1710072
x26           1710072
x27           1710072
x28           1710072
x29           1651536
x30           1651536
x31           1651536
x32           1651536
x33           1651536
x34           1651536
x35           1651536
x36           1651536
x37           1651536
x38           1651536
x39       

In [ ]:
# y값 결측치 제거
data = total[~total["y"].isna()]
print(f"y 값 결측치 제거 후: {len(total)} → {len(data)}")

# x값 결측치 제거
drop_x_features = [f"x{i}" for i in range(22, 49+1)]

data = data.drop(columns=drop_x_features, axis=1)

print(f"x22-x49 컬럼 제거 후: {data.shape}")

# 직접 지정한 x값 제거: lot_cd, oper_area

data = data.drop(columns=["lot_cd", "oper_area"], axis=1)
data.reset_index(drop=True, inplace=True)

print(f"불필요한 컬럼 제거 후: {data.shape}")

display(data.head(10))

y 값 결측치 제거 후: 1729584 → 1671457
x22-x49 컬럼 제거 후: (1671457, 29)
불필요한 컬럼 제거 후: (1671457, 27)


,timekey_hr,oper_group,oper_id,days,shift,x1,x2,x3,x4,x5,...,x13,x14,x15,x16,x17,x18,x19,x20,x21,y
0,2025050309,group1,oper1,SAT,A,PG11,2,3900,487.5,0.103,...,400,875,25,0,650,75,50,0,0,0.101
1,2025050310,group1,oper1,SAT,A,PG11,2,3900,650.0,0.077,...,450,1175,25,0,850,100,50,0,0,0.144
2,2025050311,group1,oper1,SAT,A,PG11,2,3900,812.5,0.092,...,525,1250,25,0,975,200,100,0,0,0.165
3,2025050312,group1,oper1,SAT,A,PG11,2,3900,975.0,0.128,...,550,1675,75,0,1050,275,175,0,0,0.196
4,2025050313,group1,oper1,SAT,A,PG11,2,3900,1137.5,0.132,...,525,1575,125,0,925,350,200,0,0,0.199
5,2025050314,group1,oper1,SAT,A,PG11,2,3900,1300.0,0.135,...,400,1550,175,0,875,225,350,0,0,0.200
6,2025050315,group1,oper1,SAT,B,PG11,2,3900,1462.5,0.120,...,600,1750,250,0,1150,275,250,0,0,0.248
7,2025050316,group1,oper1,SAT,B,PG11,2,3900,1625.0,0.200,...,250,1625,275,0,950,200,350,0,0,0.255
8,2025050317,group1,oper1,SAT,B,PG11,2,3900,1787.5,0.126,...,375,1800,250,0,1225,175,250,0,0,0.291
9,2025050318,group1,oper1,SAT,B,PG11,2,3900,1950.0,0.090,...,150,1650,225,0,925,225,200,0,0,0.255


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
scaler = StandardScaler()
continuous_data = data[["x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "x10", "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20", "x21"]]

continuous_data = continuous_data.replace(np.inf, 1e+5)
scaled_continuous_data = pd.DataFrame(scaler.fit_transform(continuous_data), columns=continuous_data.columns)

In [ ]:
for col in ["oper_group","days","shift","x1"]:
    print(f"{col} 컬럼의 고유값: {data[col].nunique()}")


oper_group 컬럼의 고유값: 277
days 컬럼의 고유값: 7
shift 컬럼의 고유값: 3
x1 컬럼의 고유값: 20


# 📕 데이터셋 구성 및 모델 구현
-----
* 활용하고자 하는 딥러닝 모델에 맞게 PyTorch의 Dataset과 DataLoader를 구성해야 함
    * Dataset: 모델에 적합하게 데이터를 구성해놓은 pytorch의 객체
    * DataLoader: Dataset을 원하는 크기의 배치(batch)로 구성해주는 객체
* 두 가지 모델을 활용하고자 함: Transformer, TabTransformer


In [ ]:
timekey_oper_id = dict()
max_oper_id_counts = 0
for group in data.groupby("timekey_hr"):
    timekey_oper_id[group[0]] = group[1]["oper_id"].nunique()
    max_oper_id_counts = max(max_oper_id_counts, timekey_oper_id[group[0]])


print(f"len(timekey_oper_id): {len(timekey_oper_id)}")
print(f"{list(timekey_oper_id.items())[:5]}")
print(f"max_oper_id_counts: {max_oper_id_counts}")

len(timekey_oper_id): 2136
[(2025050307, 278), (2025050308, 530), (2025050309, 605), (2025050310, 640), (2025050311, 676)]
max_oper_id_counts: 803
